In [92]:
# lets do some multiple instance learning!
#!pip install -e git+https://github.com/garydoranjr/misvm.git#egg=misvm
import misvm
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as auc_score
import numpy as np
import matplotlib.pyplot as plt
import json
import os
import glob

In [93]:
# loading a feature file
src_file = '/home/colin/results_cov/covid_class_v5_features_q7qx2q3r_fold_0.json'
with open(src_file) as fp:
    src_features = json.load(fp)

In [96]:
src_features['10049501687'].keys()

dict_keys(['train_features', 'train_predictions', 'train_label', 'train_files'])

In [97]:
# # lets load the src data -- looking at a sample
# src_file = '/home/colin/results_cov/covid_class_v5_1gjbinvl_fold_3.json'
# with open(src_file) as fp:
#     src_data = json.load(fp)
test_data = {order:data for order, data in src_features.items() if 'predictions' in data}
val_data = {order:data for order, data in src_features.items() if 'val_predictions' in data}
train_data = {order:data for order, data in src_features.items() if 'train_predictions' in data}

In [40]:
def eval_classifier(model, val_labels, val_bags, test_labels, test_bags):
    model.fit(val_bags, val_labels)
    bag_labs = model.predict(test_bags)
    multi_roc = roc_auc_score(test_labels, bag_labs)
    return multi_roc

In [99]:
roc_score

0.7833333333333333

In [98]:
# lets now train a few SVMs on the validation data
linear_misvm = misvm.MISVM(kernel='polynomial', verbose=True)
linear_mica = misvm.MICA(kernel='linear', verbose=False)
nsk = misvm.NSK(C=10, verbose=False, kernel='linear_fs')
sil = misvm.SIL(C=10, verbose=False)
miss = misvm.MissSVM(verbose=False)
classifiers = [linear_misvm, nsk]
class_names = ["Linear MISVM", 'NSK']

val_bags, val_labels = zip(*[(data['val_features'], data['val_label']) for data in val_data.values()])
test_bags, test_labels = zip(*[(data['features'], data['label']) for data in test_data.values()])
scale = 1
val_bags = [np.array(bag).reshape(-1, 64)*scale for bag in val_bags]
test_bags = [np.array(bag).reshape(-1, 64)*scale for bag in test_bags]

roc_scores = []
for classifier, classifier_name in zip(classifiers, class_names):
    roc_score = eval_classifier(classifier, np.sign(val_labels)*2 - 1, val_bags, test_labels, test_bags)
    roc_scores.append(roc_score)
for class_name, score in zip(class_names, roc_scores):
    print(class_name, score)
print('median ROC', median_roc)


Non-random start...

Iteration 1...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -1.7764e+01 -2.0091e-01  1e+05  3e+02  6e-15
 1: -2.6318e-01 -3.6633e-03  1e+03  5e+00  1e-14
 2: -3.0230e-03 -1.0776e-05  2e+01  5e-02  2e-14
 3: -3.1645e-05 -1.7383e-06  2e-01  5e-04  2e-14
 4: -1.0171e-06 -1.7292e-06  2e-03  6e-06  2e-14
 5: -6.7946e-07 -1.4380e-06  2e-04  5e-07  3e-14
 6: -6.2371e-07 -1.0934e-06  2e-06  5e-09  5e-13
 7: -6.2354e-07 -8.4139e-07  2e-07  4e-11  5e-14
 8: -6.2358e-07 -6.2609e-07  3e-09  4e-13  9e-14
Optimal solution found.
Recomputing classes...
Selector differences: 35280
Updating QP...

Iteration 2...
Training SVM...
     pcost       dcost       gap    pres   dres
 0: -9.8437e+00 -3.9728e-01  1e+05  3e+02  2e-13
 1: -5.9483e-01 -9.8568e-02  4e+03  1e+01  1e-13
 2: -3.3178e-02 -3.4555e-03  2e+02  5e-01  3e-14
 3: -1.4422e-03 -7.4535e-05  7e+00  2e-02  3e-14
 4: -8.7672e-05 -5.9817e-06  4e-01  1e-03  5e-14
 5: -1.5827e-06 -1.9466e-06  9e-03  3e-05  4

Linear MISVM 0.8920454545454546
NSK 0.40625000000000006
median ROC 0.8977272727272728


In [65]:
bag_labs = classifier.predict(test_bags)

In [66]:
bag_labs

array([ 0.35640892,  0.49998498, -1.09087131,  0.38911041, -0.25291742,
        0.48123175,  0.44515025, -0.78829505, -1.07619267,  0.46236454,
        0.39130005,  0.37191015,  0.43225734,  0.10374659,  0.10204424,
        0.46775638,  0.38056838, -0.73902295,  0.44539629, -1.74831147,
        0.39923935,  0.41548497,  0.49415713, -1.7986592 , -2.54853831,
       -2.87597323, -2.58488429])

In [19]:
bag_labs.shape

(27,)

In [21]:
len(test_labels)

27

In [36]:
multi_roc = roc_auc_score(val_labels, bag_labs)

In [37]:
multi_roc

0.95

In [24]:
median_roc = roc_auc_score(test_labels, [np.median(bag.flatten()) for bag in test_bags])

Linear MISVM 0.8920454545454546
RBF MISVM 0.8920454545454546
Quadratic MISVM 0.8920454545454546
Linear MICA 0.8920454545454546
RBF MICA 0.46590909090909094
Quadratic MICA 0.8920454545454546
median ROC 0.8977272727272728
